In [1]:
from solver import ipm_overleaf, ipm_chapter14
import gzip
import pickle

from scipy.optimize import linprog
import numpy as np
import time

In [2]:
sp_var = []
rel_err = []
abs_err = []

scp_obj = []
abs_obj_err = []
rel_obj_err = []

In [3]:
for i in range(1):
    with gzip.open(f"small_instances/raw/instance_{i}.pkl.gz", "rb") as file:
        (A, b, c) = pickle.load(file)

    c = c.numpy()
    A_ub = None
    b_ub = None
    A_eq = A.numpy()
    b_eq = b.numpy()
    bounds = None
    
    t1 = time.time()
    sol = ipm_overleaf(c, A_ub, b_ub, A_eq, b_eq, bounds, max_iter = 1000, tol = 1.e-9, lin_solver='scipy_cg')
    t2 = time.time()
    
    sol_cp = linprog(c, 
                    A_ub=A_ub, 
                    b_ub=b_ub, 
                    A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='interior-point')
    
    t3 = time.time()
    
    print(f'{i}th problem - time: ours {t2 - t1}, scipy {t3 - t2}, var gap {np.abs(sol["x"] - sol_cp.x).max()} '
          f'steps: ours {sol["nit"]} scipy {sol_cp.nit}')
    sp_var.append(sol_cp.x)
    abs_err.append(np.abs(sol["x"] - sol_cp.x))
    rel_err.append(np.abs(sol["x"] - sol_cp.x) / (sol_cp.x + 1.e-7))
    scp_obj.append(sol_cp.fun)
    abs_obj_err.append(np.abs(sol_cp.fun - sol['fun']))
    rel_obj_err.append(np.abs(sol_cp.fun - sol['fun']) / sol_cp.fun)

  0%|                                                                                                                                                         | 0/1000 [00:00<?, ?it/s]/home/chendi/ipm_gnn/solver.py:352: UserWarning: Instability occured at iter 0, turning to lstsq
  warnings.warn(f'Instability occured at iter {iteration}, turning to lstsq')
  4%|████▎                                                                                                            | 38/1000 [00:00<00:01, 645.57it/s, lin_system_steps=0, obj=6.86]

0th problem - time: ours 0.07434415817260742, scipy 0.004148960113525391, var gap 1.2508253877108189e-09 steps: ours 38 scipy 9



/tmp/ipykernel_4559/2682378014.py:16: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  sol_cp = linprog(c,


In [ ]:
sp_var = np.concatenate(sp_var, axis=0)
rel_err = np.concatenate(rel_err, axis=0)
abs_err = np.concatenate(abs_err, axis=0)

In [ ]:
scp_obj = np.array(scp_obj)
abs_obj_err = np.array(abs_obj_err)
rel_obj_err = np.array(rel_obj_err)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=False, figsize=(12, 10))
axes[0, 0].set_title('variable - rel err')
sns.scatterplot(x=sp_var, y=rel_err * 100, ax=axes[0, 0])
axes[0, 0].set_ylabel('rel err (%)')
axes[0, 0].set_xlabel('x value')
axes[0, 0].set_xscale('log')

axes[0, 1].set_title('variable - abs err')
sns.scatterplot(x=sp_var, y=abs_err, ax=axes[0, 1])
axes[0, 1].set_ylabel('abs err')
axes[0, 1].set_xlabel('x value')
axes[0, 1].set_xscale('log')

axes[1, 0].set_title('obj - rel err')
sns.scatterplot(x=scp_obj, y=rel_obj_err * 100, ax=axes[1, 0])
axes[1, 0].set_ylabel('rel err (%)')
axes[1, 0].set_xlabel('obj value')

axes[1, 1].set_title('obj - abs err')
sns.scatterplot(x=scp_obj, y=abs_obj_err, ax=axes[1, 1])
axes[1, 1].set_ylabel('abs err')
axes[1, 1].set_xlabel('obj value')

plt.savefig('subplot.png')